In [1]:
#!/usr/bin/env python -W ignore::DeprecationWarning

# General Utilities
import glob, pickle, os, time
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

# Image processing & analysis functions
from skimage.feature import hog
from skimage.util import img_as_ubyte
from skimage.color import rgb2hsv, rgb2luv, rgb2ycbcr
from skimage.exposure import rescale_intensity, equalize_adapthist
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from scipy.ndimage.measurements import label

# Import classifier
#from sklearn.svm import SVC
from xgboost import XGBClassifier

# Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Import Tracker Class for keeping track of heatmaps
from tracker import Tracker

# Supress warnings (generated primarily by xgb)
import warnings
warnings.filterwarnings("ignore")

/home/ucalegon/pythoconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Convert images from RGB to specified color_map
def rgb_convert(img, color_map):
    
    if color_map == 'HSV':
        img_convert = rgb2hsv(img)
    elif color_map == 'LUV':
        img_convert = rgb2luv(img)
    elif color_map == 'YCbCr':
        img_convert = rgb2ycbcr(img)
    
    return img_convert

# Pickle trained classifier
def dump_classifier(clf, clf_name):
    '''
    Pickle classifier using clf_name
    '''
    if '.pickle' in clf_name:
        pass
    else:
        clf_name += '.pickle'
    with open('clfs/'+clf_name, 'wb') as f:
        pickle.dump(clf, f)
    f.close()

    print('Classifier dumped to clfs/{}'.format(clf_name))

    return

# Load trained classifier
def load_clf(clf_name):
    with open('clfs/'+clf_name, 'rb') as f:
        clf = pickle.load(f)
    f.close()
    
    return clf

In [3]:
# Load pickled arrays of images & labels output from input_images.py
def load_images_labels(processed_images = False):
    if not processed_images:
        with open('clf_images/all_vehicles_{}.pickle'.format(color_map), 'rb') as f:
            all_vehicles = pickle.load(f)
        f.close()
    else:
        with open('clf_images/all_vehicles_{}_processed.pickle'.format(color_map), 'rb') as f:
            all_vehicles = pickle.load(f)
        f.close()

    with open('clf_images/all_vehicles_labels.pickle', 'rb') as f:
        all_vehicles_labels = pickle.load(f)
    f.close()

    return all_vehicles, all_vehicles_labels

In [4]:
# Generate Histogram of Oriented Gradients Features
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=False,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False,
                       visualise=vis, feature_vector=feature_vec)
        return features

# Generate spatial features
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))

# Generate color histogram features
def color_hist(img, nbins=32, bins_range = (0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [5]:
# Feature extraction method for classifier training. 
def extract_features(imgs, spatial_size=(32, 32), hist_bins=32, hist_range=(0, 256), orient=12, pix_per_cell=8, cell_per_block=2, hog_channel='ALL'):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for image in tqdm(imgs):
        feature_image = np.copy(image)

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel],
                                    orient, pix_per_cell, cell_per_block,
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins) #, bins_range=hist_range
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    # Return list of feature vectors

    feature_names = ['f_{}'.format(x) for x in range(len(features[0]))]
    features_pd = pd.DataFrame(features, columns = feature_names)



    return features_pd

In [6]:
# Train classifier
def train_clf(features, labels, clf_type = 'xgb'):
    
    # Create train/test split for validation
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 42)
    
    # Fit, Transform & Dump scaler
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # XGBoost classifier
    if clf_type == 'xgb':

        clf = XGBClassifier()
        clf_final = XGBClassifier()

    # Gridsearch over Support Vector Machines using linear and polynomial kernels
    elif clf_type == 'svc':

        params = {
                'kernel': ('linear', 'poly')
                }

        svc = SVC()
        clf = GridSearchCV(svc, param_grid = params)
        clf_final = GridSearchCV(svc, param_grid = params_)

    # Train clf
    t_0 = time.time()
    print('training for validation...')
    clf.fit(X_train_scaled, y_train)
    t_1 = time.time()
    print('{} clf fit in {:.2f} s'.format(clf_type, t_1-t_0))
    
    # Predict and report clf quality
    y_test_hat = clf.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_test_hat)
    f1 = f1_score(y_test, y_test_hat)
    clf_report = classification_report(y_test, y_test_hat)

    print('Accuracy = {}'.format(accuracy))
    print('f1 Score = {}'.format(f1))
    print('\n')
    print(clf_report)
    
    # After validation, train on whole training set
    
    scaler_final = StandardScaler().fit(features)
    features_scaled = scaler_final.transform(features)
    t_2 = time.time()
    print('training for final...')
    clf_final.fit(features_scaled, labels)
    t_3 = time.time()
    print('{} final clf fit in {:.2f} s'.format(clf_type, t_3-t_2))
    
    scaler_name = 'scaler_{}.pickle'.format(color_map)
    dump_classifier(scaler_final, scaler_name)

    return clf_final


In [7]:
# Pipeline for training classifier
def train_clf_pipeline():
    images, labels = load_images_labels(processed_images = processed_images)
    
    # If pickled features array exists, load; else, create
    if os.path.isfile('clf_images/all_vehicles_{}_features.pickle'.format(color_map)):
        features_pd = pd.read_pickle('clf_images/all_vehicles_{}_features.pickle'.format(color_map))
        print('features loaded from pickle')
    else:
        features_pd = extract_features(images)
        features_pd.to_pickle('clf_images/all_vehicles_{}_features.pickle'.format(color_map))
        print('features dumped to all_vehicles_{}_features.pickle'.format(color_map))

    # Train and save classifier
    clf = train_clf(features_pd, labels, clf_type = clf_type)
    clf_name = '{}_{}_{}.pickle'.format(clf_type, color_map, time.strftime('%m-%d-%H-%M'))
    dump_classifier(clf, clf_name)
    
    return


In [8]:
# Define a single function that can extract features using hog sub-sampling and make predictions
# Return windows wherein a car is detected
def find_windows(img, ystart, ystop, scale, clf, scaler, orient, pix_per_cell, cell_per_block, cells_per_step, spatial_size, hist_bins):
    # Ensure image is scaled between (0,255)
    img = img_as_ubyte(img)
    
    windows_hat = []
    
    # Restrit image to area of interest
    img_tosearch = rescale_intensity(img[ystart:ystop,:,:])
    # Convert colorspace given color_map parameter
    if color_map != 'RGB':
        ctrans_tosearch = rgb_convert(img_tosearch, color_map)
    # Scale image to search over different window sizes
    # scale = 1; window = 64x64
    # scale = 1.5; window = 96x96
    # scale = 2; window = 128x128
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1
    nfeat_per_block = orient*cell_per_block**2

    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1

    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step

    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel()
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            test_prediction = clf.predict(test_features)
            
            # If the prediction is True, add window to windows_hat
            if test_prediction[0] == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                
                window_i = ((xbox_left, ytop_draw+ystart), (xbox_left+win_draw, ytop_draw+win_draw+ystart))
                windows_hat.append(window_i)
                
    return windows_hat


In [9]:
# Draw bounding boxes on image
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=4):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy
    
# Add heat to pixels within bboxes of bbox_list
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

# Apply threshold to heatmap
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap < threshold] = 0
    # Return thresholded map
    return heatmap

# Draw labeled bounding boxes on image
# given output from scipy labels function which identifies distinct areas of rectangles
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 4)
    # Return the image
    return img


In [10]:
def vehicle_tracker_pipeline(frame):
    
    # define smoothing factor
    n = smth_fctr
    
    # Initialize list of windows (bboxes) wherein a car was detected
    windows_hat = []
    
    for scale in scales_dict.keys():
        # Use different ystart, ystop & cells_per_step depending on scale (i.e. window size)
        ystart = scales_dict[scale][0]
        ystop = scales_dict[scale][1]
        cells_per_step = scales_dict[scale][2]
        # Find windows and append to windows_hat
        windows_hat_i = find_windows(frame, ystart, ystop, scale, clf, scaler, orient, pix_per_cell, cell_per_block, cells_per_step, spatial_size, hist_bins)
        for i in windows_hat_i:
            windows_hat.append(i)

    # Generate heatmap given bounding boxes (windows_hat) wherein a car was detected
    heatmap = np.zeros(frame.shape[:2])
    heatmap = add_heat(heatmap, windows_hat)
    
    # Add heatmap to array of recent heatmaps for smoothing
    car_tracker.recent_heatmaps.append(heatmap)
    car_tracker.recent_heatmaps = car_tracker.recent_heatmaps[-n:]
    heatmap_avg = np.average(car_tracker.recent_heatmaps, axis = 0)
    
    # Apply threshold to heatmap
    heatmap_thresh = apply_threshold(heatmap_avg, threshold = heat_thresh)

    # Use labels function from scipy to detect separate vehicles
    labels = label(heatmap_thresh)
    if labels[1] == 0:
        car_tracker.recent_heatmaps = []
    
    # Generate output image by drawing bounding boxes (via labels) on copy of image
    out_img = draw_labeled_bboxes(np.copy(frame), labels)
    
    return out_img

    

In [11]:
### PARAMETERS ###

# High level parameters defining I/O
testing = False
processed_images = False
# Color Map Choices: RGB, HSV, LUV, YCbCr
color_map = 'YCbCr'
clf_type = 'xgb'

# Parameters for Histogram of Oriented Gradients features
# scales_dict format :: scale: (ystart, ystop, cells_per_step)
scales_dict = {
    1: (400, 464, 1),
    1.5: (400, 528, 1),
    2: (400, 592, 2)
}
orient=12
pix_per_cell=8
cell_per_block=2


# Parameter for bin spatial feature
spatial_size = (32, 32)

# Parameters for color histogram feature
hist_bins=32
hist_range=(0, 256)

# Heatmap threshold
heat_thresh = 1.75

# Smoothing Factor
smth_fctr = 12


In [12]:
train_clf_pipeline()

# Load scaler and classifier from train_set trained images 
scaler = load_clf('scaler_{}.pickle'.format(color_map))
clf = load_clf('{}_{}.pickle'.format(clf_type, color_map))

features loaded from pickle
training for validation...
xgb clf fit in 516.24 s
Accuracy = 0.9923986486486487
f1 Score = 0.9922346850733391


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      1801
        1.0       1.00      0.99      0.99      1751

avg / total       0.99      0.99      0.99      3552

training for final...
xgb final clf fit in 653.16 s
Classifier dumped to clfs/scaler_YCbCr.pickle
Classifier dumped to clfs/xgb_YCbCr_06-29-12-42.pickle


In [13]:
output_name = 'result.mp4'
suboutput_name = 'subclip_result.mp4'

clip = VideoFileClip("project_video.mp4")

In [14]:
# If testing, process subclip
if testing:

    global car_tracker
    car_tracker = Tracker()
    
    subclip = clip.subclip(5, 15)
    suboutput = subclip.fl_image(vehicle_tracker_pipeline)
    %time suboutput.write_videofile(suboutput_name, audio=False)

In [15]:
# If not testing, process whole clip
if not testing:

    global car_tracker
    car_tracker = Tracker()

    output = clip.fl_image(vehicle_tracker_pipeline) 
    %time output.write_videofile(output_name, audio=False)

[MoviePy] >>>> Building video result.mp4
[MoviePy] Writing video result.mp4


100%|█████████▉| 1260/1261 [1:33:42<00:04,  4.61s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result.mp4 

CPU times: user 5h 34min 10s, sys: 7min 41s, total: 5h 41min 52s
Wall time: 1h 33min 44s
